# Natural Language Processing 
### Exercise III

Gillie S. Calanuga & Michael Rey T. Tuando <br>
BSCS 3A

In [1]:
# Import Necessary Libraries
import wikipedia
import re
import math
import nltk
from nltk import bigrams, trigrams
from nltk.tokenize import word_tokenize
from collections import Counter

# Download necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Michael
[nltk_data]     Rey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1.	 (10 points) Use the Wikipedia python module and access any topic, as you will use that as your corpus, with a word limit of 1000 words.

In [2]:
# Accessing Wikipedia topic

topic = "EDSA People Power"

# Get the content of a Wikipedia page

page = wikipedia.page(topic)
text = page.content[:1000]

# Remove punctuation and convert to lowercase letters
text = re.sub(r'[^\w\s]', '', text).lower() 

# Tokenize the Words
tokens = word_tokenize(text)

# Print the cleaned content of the page
print(text)

the people power revolution also known as the edsa revolution or the february revolution were a series of popular demonstrations in the philippines mostly in metro manila from february 22 to 25 1986 there was a sustained campaign of civil resistance against regime violence and electoral fraud the nonviolent revolution led to the departure of ferdinand marcos the end of his 20year dictatorship and the restoration of democracy in the philippines
it is also referred to as the yellow revolution due to the presence of yellow ribbons during demonstrations in reference to the tony orlando and dawn song tie a yellow ribbon round the ole oak tree as a symbol of protest following the assassination of filipino senator benigno ninoy aquino jr in august 1983 upon his return to the philippines from exile  it was widely seen as a victory of the people against two decades of presidential rule by president marcos and made news headlines as the revolution that surprised the world
t


2.	(60 points) Train 2 models: a Bigram and Trigram Language Model, use the shared code as reference for bigram modeling, and update it to support trigrams.

In [3]:
# Bigram Probability Models with Laplace Smoothing

def bigram_probabilities(tokens, alpha=1):
    bigram_counts = Counter(bigrams(tokens))
    unigram_counts = Counter(tokens)
    
    vocab_size = len(unigram_counts)  # Vocabulary size for smoothing
    bigram_probs = {
        bigram: (count + alpha) / (unigram_counts[bigram[0]] + alpha * vocab_size)
        for bigram, count in bigram_counts.items()
    }
    return bigram_probs

print ("Bigram Probabilities:",bigram_probabilities(tokens))

Bigram Probabilities: {('the', 'people'): 0.02564102564102564, ('people', 'power'): 0.019801980198019802, ('power', 'revolution'): 0.02, ('revolution', 'also'): 0.01904761904761905, ('also', 'known'): 0.019801980198019802, ('known', 'as'): 0.02, ('as', 'the'): 0.038461538461538464, ('the', 'edsa'): 0.017094017094017096, ('edsa', 'revolution'): 0.02, ('revolution', 'or'): 0.01904761904761905, ('or', 'the'): 0.02, ('the', 'february'): 0.017094017094017096, ('february', 'revolution'): 0.019801980198019802, ('revolution', 'were'): 0.01904761904761905, ('were', 'a'): 0.02, ('a', 'series'): 0.019230769230769232, ('series', 'of'): 0.02, ('of', 'popular'): 0.01834862385321101, ('popular', 'demonstrations'): 0.02, ('demonstrations', 'in'): 0.0297029702970297, ('in', 'the'): 0.028846153846153848, ('the', 'philippines'): 0.03418803418803419, ('philippines', 'mostly'): 0.0196078431372549, ('mostly', 'in'): 0.02, ('in', 'metro'): 0.019230769230769232, ('metro', 'manila'): 0.02, ('manila', 'from'): 

In [4]:
# Trigram Probability Models with Laplace Smoothing

def trigram_probabilities(tokens, alpha=1):
    trigram_counts = Counter(trigrams(tokens))
    bigram_counts = Counter(zip(tokens, tokens[1:]))
    
    vocab_size = len(bigram_counts)  # Vocabulary size for smoothing
    trigram_probs = {
        trigram: (count + alpha) / (bigram_counts[(trigram[0], trigram[1])] + alpha * vocab_size)
        for trigram, count in trigram_counts.items()
    }
    return trigram_probs

print ("Trigram Probabilities:",trigram_probabilities(tokens))


Trigram Probabilities: {('the', 'people', 'power'): 0.012987012987012988, ('people', 'power', 'revolution'): 0.013071895424836602, ('power', 'revolution', 'also'): 0.013071895424836602, ('revolution', 'also', 'known'): 0.013071895424836602, ('also', 'known', 'as'): 0.013071895424836602, ('known', 'as', 'the'): 0.013071895424836602, ('as', 'the', 'edsa'): 0.012903225806451613, ('the', 'edsa', 'revolution'): 0.013071895424836602, ('edsa', 'revolution', 'or'): 0.013071895424836602, ('revolution', 'or', 'the'): 0.013071895424836602, ('or', 'the', 'february'): 0.013071895424836602, ('the', 'february', 'revolution'): 0.013071895424836602, ('february', 'revolution', 'were'): 0.013071895424836602, ('revolution', 'were', 'a'): 0.013071895424836602, ('were', 'a', 'series'): 0.013071895424836602, ('a', 'series', 'of'): 0.013071895424836602, ('series', 'of', 'popular'): 0.013071895424836602, ('of', 'popular', 'demonstrations'): 0.013071895424836602, ('popular', 'demonstrations', 'in'): 0.013071895

In [5]:
# Compute probability distributions

wiki_bigram = bigram_probabilities(tokens)
wiki_trigram = trigram_probabilities(tokens)

In [6]:
# Test Sentence
test_sentence = "The EDSA People Power Revolution was a peaceful uprising in 1986 that led to the restoration of democracy in the Philippines"
print("Test Sentence:", test_sentence)

Test Sentence: The EDSA People Power Revolution was a peaceful uprising in 1986 that led to the restoration of democracy in the Philippines


3.	(30 points) Using a test sentence “The quick brown fox jumps over the lazy dog near the bank of the river.” OR generate your own test sentence, create a function that will determine the perplexity score for each trained model.

In [7]:
# Bigram Perplexity Computation using Log Probabilities

def calculate_bigram_perplexity(bigram_probs, test_sentence, alpha=1):
    test_tokens = word_tokenize(test_sentence.lower())
    test_bigrams = list(bigrams(test_tokens))
    
    N = len(test_bigrams)
    log_prob_sum = 0
    vocab_size = len(set(test_tokens))  # Use test set vocab size for smoothing
    
    for bigram in test_bigrams:
        prob = bigram_probs.get(bigram, alpha / (1 + alpha * vocab_size))  # Laplace smoothing for unknown bigrams
        log_prob_sum += math.log2(prob)
    
    return math.pow(2, -log_prob_sum / N) if N > 0 else float('inf')  # Avoid division by zero

print ("Calculate Bigram Probabilities:",calculate_bigram_perplexity(wiki_bigram,test_sentence))


Calculate Bigram Probabilities: 31.737510186168404


In [8]:
# Trigram Perplexity Computation using Log Probabilities

def calculate_trigram_perplexity(trigram_probs, test_sentence, alpha=1):
    test_tokens = word_tokenize(test_sentence.lower())
    test_trigrams = list(trigrams(test_tokens))
    
    N = len(test_trigrams)
    log_prob_sum = 0
    vocab_size = len(set(test_tokens))
    
    for trigram in test_trigrams:
        prob = trigram_probs.get(trigram, alpha / (1 + alpha * vocab_size))  # Laplace smoothing
        log_prob_sum += math.log2(prob)
    
    return math.pow(3, -log_prob_sum / N) if N > 0 else float('inf')  # Avoid division by zero

print ("Calculate Trigram Probabilities:",calculate_trigram_perplexity(wiki_trigram,test_sentence))


Calculate Trigram Probabilities: 232.03266711889697


In [9]:
# Compute perplexities

bigram_perplexity_score = calculate_bigram_perplexity(wiki_bigram, test_sentence)
trigram_perplexity_score = calculate_trigram_perplexity(wiki_trigram, test_sentence)

In [10]:
# Print results

print("Bigram Model Perplexity -> "f'{test_sentence}')
print("Score:", bigram_perplexity_score)
print("Trigram Model Perplexity -> "f'{test_sentence}')
print("Score:", trigram_perplexity_score)

Bigram Model Perplexity -> The EDSA People Power Revolution was a peaceful uprising in 1986 that led to the restoration of democracy in the Philippines
Score: 31.737510186168404
Trigram Model Perplexity -> The EDSA People Power Revolution was a peaceful uprising in 1986 that led to the restoration of democracy in the Philippines
Score: 232.03266711889697
